## polars example notebook
- polars の使い方を学ぶ

In [1]:
%load_ext lab_black

In [2]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow
from datetime import datetime

### 基本的な使い方

#### DataFrame の作成

In [3]:
df = pl.DataFrame(
    {
        "col_str": ["a", "b", "c"],
        "col_int": [1, None, 3],
        "col_float": [0.1, 0.2, 0.3],
        "col_bool": [True, True, False],
    }
)

print(df)

shape: (3, 4)
┌─────────┬─────────┬───────────┬──────────┐
│ col_str ┆ col_int ┆ col_float ┆ col_bool │
│ ---     ┆ ---     ┆ ---       ┆ ---      │
│ str     ┆ i64     ┆ f64       ┆ bool     │
╞═════════╪═════════╪═══════════╪══════════╡
│ a       ┆ 1       ┆ 0.1       ┆ true     │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ null    ┆ 0.2       ┆ true     │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       ┆ 0.3       ┆ false    │
└─────────┴─────────┴───────────┴──────────┘


#### 列の追加

In [4]:
df["col_bool"] = [True, True, False]

In [5]:
# Series を渡すこともできる：
df = df.with_column(pl.Series("col_bool", [True, True, False]))

#### 列名の変更

In [6]:
df = df.rename({"col_float": "col_flt"})
df

col_str,col_int,col_flt,col_bool
str,i64,f64,bool
"""a""",1,0.1,true
"""b""",null,0.2,true
"""c""",3,0.3,false


#### 型のcast

In [7]:
df["col_flt"] = df["col_flt"].cast(pl.Float32)
df

col_str,col_int,col_flt,col_bool
str,i64,f32,bool
"""a""",1,0.10000000149011612,true
"""b""",null,0.20000000298023224,true
"""c""",3,0.30000001192092896,false


#### 列の削除

In [8]:
df = df.drop("col_flt")
df

col_str,col_int,col_bool
str,i64,bool
"""a""",1,true
"""b""",null,true
"""c""",3,false


#### NULL を埋める

In [9]:
df = df.fill_null(0)
df

col_str,col_int,col_bool
str,i64,bool
"""a""",1,true
"""b""",0,true
"""c""",3,false


#### その他

In [10]:
print(df.head(2))

shape: (2, 3)
┌─────────┬─────────┬──────────┐
│ col_str ┆ col_int ┆ col_bool │
│ ---     ┆ ---     ┆ ---      │
│ str     ┆ i64     ┆ bool     │
╞═════════╪═════════╪══════════╡
│ a       ┆ 1       ┆ true     │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 0       ┆ true     │
└─────────┴─────────┴──────────┘


In [11]:
print(df[1:])

shape: (2, 3)
┌─────────┬─────────┬──────────┐
│ col_str ┆ col_int ┆ col_bool │
│ ---     ┆ ---     ┆ ---      │
│ str     ┆ i64     ┆ bool     │
╞═════════╪═════════╪══════════╡
│ b       ┆ 0       ┆ true     │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       ┆ false    │
└─────────┴─────────┴──────────┘


In [12]:
print(df[[2], [1]])

shape: (1, 1)
┌─────────┐
│ col_int │
│ ---     │
│ i64     │
╞═════════╡
│ 3       │
└─────────┘


In [13]:
print(df[-1, "col_bool"])

False


#### pandas や numpy への変換

In [14]:
print(df.to_pandas())

  col_str  col_int  col_bool
0       a        1      True
1       b        0      True
2       c        3     False


In [15]:
print(df.to_numpy())

[['a' 1 True]
 ['b' 0 True]
 ['c' 3 False]]


#### Config

In [16]:
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(10)

polars.cfg.Config

#### map や apply

In [17]:
df["col_int_div_2"] = df["col_int"].apply(lambda x: x / 2)
df = df.hstack(
    df[
        pl.when(pl.col("col_int_div_2") >= 1)
        .then(1)
        .otherwise(pl.Series([7, 8, 9]))
        .alias("wto")
    ]
)
df = df.hstack(
    df.select([pl.col("col_int").is_in([1, 2]).is_not().alias("col_int_not_in_1_2")])
)
print(df)

shape: (3, 6)
┌─────────┬─────────┬──────────┬───────────────┬─────┬────────────────────┐
│ col_str ┆ col_int ┆ col_bool ┆ col_int_div_2 ┆ wto ┆ col_int_not_in_1_2 │
│ ---     ┆ ---     ┆ ---      ┆ ---           ┆ --- ┆ ---                │
│ str     ┆ i64     ┆ bool     ┆ f64           ┆ i64 ┆ bool               │
╞═════════╪═════════╪══════════╪═══════════════╪═════╪════════════════════╡
│ a       ┆ 1       ┆ true     ┆ 0.5           ┆ 7   ┆ false              │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 0       ┆ true     ┆ 0.0           ┆ 8   ┆ true               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       ┆ false    ┆ 1.5           ┆ 1   ┆ true               │
└─────────┴─────────┴──────────┴───────────────┴─────┴────────────────────┘


#### 結合

In [18]:
df_2 = pl.DataFrame(
    {
        "col_str": ["a", "c", "d"],
        "col_datetime": [
            datetime.strptime(f"2021-10-{i} 10:20:30 +0900", "%Y-%m-%d %H:%M:%S %z")
            for i in [12, 15, 17]
        ],
    }
)
df_join = df[["col_str", "col_int"]].join(df_2, on="col_str", how="left")
print(df_join)

shape: (3, 3)
┌─────────┬─────────┬─────────────────────┐
│ col_str ┆ col_int ┆ col_datetime        │
│ ---     ┆ ---     ┆ ---                 │
│ str     ┆ i64     ┆ datetime            │
╞═════════╪═════════╪═════════════════════╡
│ a       ┆ 1       ┆ 2021-10-12 01:20:30 │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 0       ┆ null                │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       ┆ 2021-10-15 01:20:30 │
└─────────┴─────────┴─────────────────────┘


/tmp/Python/python-3.9.6/lib/python3.9/site-packages/polars/utils.py:29: UserWarning: Conversion of (potentially) timezone aware to naive datetimes. TZ information may be lost
  warnings.warn(


In [19]:
df = df[["col_str", "col_int"]].vstack(
    pl.DataFrame(
        {
            "col_str": ["x", "y", "z"],
            "col_int": [7, 8, 9],
        }
    )
)
print(df)

shape: (6, 2)
┌─────────┬─────────┐
│ col_str ┆ col_int │
│ ---     ┆ ---     │
│ str     ┆ i64     │
╞═════════╪═════════╡
│ a       ┆ 1       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 0       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ x       ┆ 7       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ y       ┆ 8       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ z       ┆ 9       │
└─────────┴─────────┘


#### フィルターとソート

In [20]:
df = df.filter((pl.col("col_int") >= 1) & (pl.col("col_int") <= 7))
df = df.sort("col_int", reverse=True)
print(df)

shape: (3, 2)
┌─────────┬─────────┐
│ col_str ┆ col_int │
│ ---     ┆ ---     │
│ str     ┆ i64     │
╞═════════╪═════════╡
│ x       ┆ 7       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 1       │
└─────────┴─────────┘


#### シフト

In [21]:
df["col_int_shifted"] = df["col_int"].shift(1)
print(df)

shape: (3, 3)
┌─────────┬─────────┬─────────────────┐
│ col_str ┆ col_int ┆ col_int_shifted │
│ ---     ┆ ---     ┆ ---             │
│ str     ┆ i64     ┆ i64             │
╞═════════╪═════════╪═════════════════╡
│ x       ┆ 7       ┆ null            │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3       ┆ 7               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 1       ┆ 3               │
└─────────┴─────────┴─────────────────┘


#### 集約

In [22]:
df = pl.DataFrame(
    {
        "col_str": ["a", "b", "c", "a", "b", "c", "a", "b", "a", "c"],
        "col_int": [1, 3, 2, 6, 5, 3, 1, 4, 2, 1],
        "col_float": [0.2, 0.4, 0.1, 0.5, 0.6, 0.8, 0.9, 0.1, 0.5, 0.2],
    }
)

#### describe

In [23]:
print(df.describe())

shape: (5, 4)
┌──────────┬─────────┬────────────────────┬─────────────────────┐
│ describe ┆ col_str ┆ col_int            ┆ col_float           │
│ ---      ┆ ---     ┆ ---                ┆ ---                 │
│ str      ┆ str     ┆ f64                ┆ f64                 │
╞══════════╪═════════╪════════════════════╪═════════════════════╡
│ mean     ┆ null    ┆ 2.8                ┆ 0.43000000000000005 │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ std      ┆ null    ┆ 1.7511900715418263 ┆ 0.28303906287138375 │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ min      ┆ null    ┆ 1                  ┆ 0.1                 │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ max      ┆ null    ┆ 6                  ┆ 0.9                 │
├╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ median   ┆ null    ┆ 2.5                ┆ 0.45                │
└──────────┴─────────┴────────────────────┴───────────────────

#### groupby

In [24]:
print(df.groupby("col_str").max())

shape: (3, 3)
┌─────────┬─────────────┬───────────────┐
│ col_str ┆ col_int_max ┆ col_float_max │
│ ---     ┆ ---         ┆ ---           │
│ str     ┆ i64         ┆ f64           │
╞═════════╪═════════════╪═══════════════╡
│ a       ┆ 6           ┆ 0.9           │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 5           ┆ 0.6           │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 3           ┆ 0.8           │
└─────────┴─────────────┴───────────────┘


In [25]:
print(df.groupby("col_str").agg({"col_int": "min"}))

shape: (3, 2)
┌─────────┬─────────────┐
│ col_str ┆ col_int_min │
│ ---     ┆ ---         │
│ str     ┆ i64         │
╞═════════╪═════════════╡
│ c       ┆ 1           │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 1           │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 3           │
└─────────┴─────────────┘


In [26]:
print(
    df.groupby("col_str").agg(
        [
            pl.col("col_float").sum(),
            pl.sum("col_int"),  # 短く書ける
            pl.sum("col_int").alias("int_sum"),  # 列名を自分でつけられる
            pl.col("col_int").list(),  # list にもできる
            pl.col("col_int").first(),  # 他にも count, mean, などなど
            (pl.col("col_int") > 2).sum().alias("col_int_gt_2_count"),  # 条件を満たすものをカウント
        ]
    )
)

shape: (3, 7)
┌─────────┬────────────────┬─────────────┬─────────┬───────────────┬───────────────┬───────────────┐
│ col_str ┆ col_float_sum  ┆ col_int_sum ┆ int_sum ┆ col_int_agg_l ┆ col_int_first ┆ col_int_gt_2_ │
│ ---     ┆ ---            ┆ ---         ┆ ---     ┆ ist           ┆ ---           ┆ count         │
│ str     ┆ f64            ┆ i64         ┆ i64     ┆ ---           ┆ i64           ┆ ---           │
│         ┆                ┆             ┆         ┆ list [i64]    ┆               ┆ u32           │
╞═════════╪════════════════╪═════════════╪═════════╪═══════════════╪═══════════════╪═══════════════╡
│ a       ┆ 2.1            ┆ 10          ┆ 10      ┆ [1, 6, ... 2] ┆ 1             ┆ 1             │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 1.1            ┆ 6           ┆ 6       ┆ [2, 3, 1]     ┆ 2             ┆ 1             │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌

#### Window 関数

In [27]:
print(
    df.select(
        [
            "col_str",
            "col_int",
            "col_float",
            pl.col("col_int").max().over("col_str").alias("max_int_by_str"),
            pl.col("col_float").mean().over("col_str").alias("avg_float_by_str"),
        ]
    )
)

shape: (10, 5)
┌─────────┬─────────┬───────────┬────────────────┬────────────────────┐
│ col_str ┆ col_int ┆ col_float ┆ max_int_by_str ┆ avg_float_by_str   │
│ ---     ┆ ---     ┆ ---       ┆ ---            ┆ ---                │
│ str     ┆ i64     ┆ f64       ┆ i64            ┆ f64                │
╞═════════╪═════════╪═══════════╪════════════════╪════════════════════╡
│ a       ┆ 1       ┆ 0.2       ┆ 6              ┆ 0.525              │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 3       ┆ 0.4       ┆ 5              ┆ 0.3666666666666667 │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ c       ┆ 2       ┆ 0.1       ┆ 3              ┆ 0.3666666666666667 │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 6       ┆ 0.5       ┆ 6              ┆ 0.525              │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 5       ┆ 0.6       ┆ 5              

In [28]:
# overの中身でsortしておかないと上手くいかない
print(
    df.sort("col_str").select(
        [
            "col_str",
            "col_int",
            "col_float",
            pl.col("col_int")
            .rank("min")
            .over("col_str")
            .flatten()
            .alias("rank_int_by_str"),
        ]
    )
)

shape: (10, 4)
┌─────────┬─────────┬───────────┬─────────────────┐
│ col_str ┆ col_int ┆ col_float ┆ rank_int_by_str │
│ ---     ┆ ---     ┆ ---       ┆ ---             │
│ str     ┆ i64     ┆ f64       ┆ u32             │
╞═════════╪═════════╪═══════════╪═════════════════╡
│ a       ┆ 1       ┆ 0.2       ┆ 1               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 6       ┆ 0.5       ┆ 4               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 1       ┆ 0.9       ┆ 1               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ a       ┆ 2       ┆ 0.5       ┆ 3               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 3       ┆ 0.4       ┆ 1               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 5       ┆ 0.6       ┆ 3               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ b       ┆ 4       ┆ 0.1       ┆ 2               │
├╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌